In [12]:
%run StudyHelperFile.ipynb


Variables: my_trade
           aapl_put_df
           date_string_list
           date_datetime_list
           complete_c
           my_verticle
           
Function: add_days_to_date



### This will iterate through all the possible trades that met the critera and only exit when DTE has expired

In [11]:
made_trade = pd.DataFrame(columns = ['STRIKE','TRADE_DATE','TRADE_LAST_PRICE','DTE','EXIT_DATE','EXIT_LAST_PRICE','EXPIRE_DATE','AMOUNT_REC','AMOUNT_BUYBACK','PROFIT_PERCENT','PROFIT'])
next_closest_trade_string = complete_c[0]['QUOTE_DATE'][0]

for i in range(len(complete_c)):
    each_df = complete_c[i]
    # the very first row of the df is our trade and data below that is the changed value due to time.
    # next closest trade date has to match the first row of the df since that is our original trade date
    if next_closest_trade_string == each_df['QUOTE_DATE'][0]:
        amount_received = ((each_df['P_ASK'][0] + each_df['P_BID'][0]) / 2)
        amount_buyback = ((each_df['P_ASK'].tail(1).tolist()[0] + each_df['P_BID'].tail(1).tolist()[0]) / 2)
        profit = (amount_received - amount_buyback).round(3)
        profit_percent = (((amount_received - amount_buyback).round(3)) / amount_received) * 100 

        next_date = each_df['QUOTE_DATE'][0]
        dte = each_df['DTE'][0]
        date_expired = add_days_to_date(next_date, dte)
        # the loop with end if there is no more data to work with which is when date_expired exceeds the last value of our date list
        if date_expired > date_datetime_list[-1]:
            break

        # This will use the date expired(when we exited the trade) and find the next closest trade date in the date list
        next_closest_trade_compare = min([ y for y in date_datetime_list if y >= date_expired], key=lambda j:abs(j-date_expired))
        next_closest_trade_string = next_closest_trade_compare.strftime('%Y-%m-%d')
        
        single_made_trade = pd.DataFrame([{'TRADE_DATE' : each_df['QUOTE_DATE'][0],
                                        'TRADE_LAST_PRICE': each_df['LAST_PRICE'][0],
                                        'EXIT_DATE': each_df['QUOTE_DATE'].tail(1).tolist()[0],
                                        'EXIT_LAST_PRICE': each_df['LAST_PRICE'].tail(1).tolist()[0],
                                        'STRIKE' : each_df['STRIKE'][0],
                                        'EXPIRE_DATE' : each_df['EXPIRE_DATE'][0], 
                                        'DTE' : each_df['DTE'][0], 
                                        'AMOUNT_REC' : amount_received, 
                                        'AMOUNT_BUYBACK' : amount_buyback, 
                                        'PROFIT_PERCENT' : profit_percent, 
                                        'PROFIT': profit}])
        made_trade = pd.concat([made_trade, single_made_trade])
        made_trade = made_trade.reset_index(drop= True)

made_trade

,STRIKE,TRADE_DATE,TRADE_LAST_PRICE,DTE,EXIT_DATE,EXIT_LAST_PRICE,EXPIRE_DATE,AMOUNT_REC,AMOUNT_BUYBACK,PROFIT_PERCENT,PROFIT
0,112.00,2017-01-03,116.14,38.00,2017-02-10,132.12,2017-02-10,1.995,0.005,99.749373,1.990
1,132.00,2017-02-14,134.93,44.96,2017-03-31,143.67,2017-03-31,1.720,0.000,100.000000,1.720
2,140.00,2017-04-04,144.75,45.00,2017-05-19,152.97,2017-05-19,2.315,0.005,99.784017,2.310
3,150.00,2017-05-25,153.91,43.00,2017-07-07,144.17,2017-07-07,1.915,5.800,-202.872063,-3.885
4,140.00,2017-07-10,145.08,39.00,2017-08-18,157.51,2017-08-18,2.465,0.005,99.797160,2.460
5,155.00,2017-08-25,159.86,42.00,2017-10-06,155.29,2017-10-06,2.950,0.005,99.830508,2.945
6,150.00,2017-10-12,156.01,43.04,2017-11-24,174.97,2017-11-24,2.510,0.005,99.800797,2.505
7,167.50,2017-11-30,171.75,43.00,2018-01-12,177.09,2018-01-12,2.630,0.005,99.809886,2.625
8,170.00,2018-01-16,176.19,45.00,2018-03-02,176.21,2018-03-02,3.345,0.005,99.850523,3.340
9,170.00,2018-03-02,176.21,41.96,2018-04-13,174.73,2018-04-13,2.910,0.005,99.828179,2.905


In [49]:
naked_put_sell_dte_study = made_trade.copy()

In [50]:
naked_put_sell_dte_study.to_csv('naked_put_sell_dte_study.csv',index=False)